In [1]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm

periods = pd.read_csv("../distance_data/Periods.csv", header=None)
Pt = pd.read_csv("../distance_data/Pt_formatted.csv", header=0)
Rt = pd.read_csv("../distance_data/Rt_formatted.csv", header=0)
Rm = pd.read_csv("../distance_data/Rm.csv", header=None)
Rf = pd.read_csv("../distance_data/Rf.csv", header=None)
Vt = pd.read_csv("../distance_data/Vt.csv", header=None)
ticker2 = pd.read_csv("../distance_data/ticker2.csv", header=None)
ticker_b = pd.read_csv("../distance_data/ticker_b.csv", header=None)

In [2]:
Rt.head(4)

,UN,1005945D,162007Q,RDPL,BGG,SKY,GOSHA,HSY,NMK,WLB,...,MNST,O,JBHT,BXLT,ENDP,EQIX,1436513D,1431816D,SPGI,1448062D
0,0.007224,0.006211,-0.015574,-0.001657,-0.007634,-0.008546,0.007139,-0.009906,-0.008846,0.00000,...,NaN,NaN,-0.011626,NaN,NaN,NaN,0.012016,-0.022216,-0.004367,NaN
1,-0.018667,-0.018517,-0.044305,-0.011612,-0.023079,-0.008620,-0.021273,-0.019991,-0.017859,0.00000,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,-0.022721,-0.004386,NaN
2,0.011713,0.006289,0.043046,0.006709,-0.007875,0.008695,0.014493,0.020398,0.009087,0.00000,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.003958,0.023249,0.000000,NaN
3,-0.002902,0.000000,-0.012698,0.003337,-0.003954,-0.008620,0.000000,0.013333,0.000000,-0.01031,...,NaN,NaN,-0.035288,NaN,NaN,NaN,-0.023653,-0.034118,0.000000,NaN


In [3]:
days, num_assets = np.shape(Rt)

print(f"Days: {days} | Assets: {num_assets}")

daylag = 0
wi_update = 1
years = 2015 - 1990 + 0.5

Days: 6425 | Assets: 1100


In [4]:
# Defining parameters for analysis

no_pairs = 5 # number of pairs to be selected and used in operations
trading_costs = 0 # absolute trading cost
percentage_costs = 0.002 # buy/sell (percentage cost for opening and closing pairs: 0.001, 0.002, for example)
trade_req = 0 # set whether (0) or not (2) positive trading volume is required for opening/closing a pair
Stop_loss = 0.95 # Choose how much loss we are willing to accept on a given pair, compared to 1, i.e, 0.93 = 7% stop loss

if Stop_loss != float('-inf'):
    stop_dir = 100 - (Stop_loss * 100)
Stop_gain = float('inf') # Choose how much gain we are willing to accept on a given pair, compared to 1, i.e 1.10 = 10% stop gain, set float('inf') if no limit for gain
s1218 = 1  # listing req. (look ahead): 12+6 months (=1)

opening_threshold = 1.5
closing_threshold = 0.75

duration_limit = float('inf')

In [5]:
avg_price_dev = np.zeros(
    (days-sum(periods.iloc[0:2, 0].to_list()), no_pairs*2))
# 12 months are w/o price deviations. The first 12 months are formation period

# Creating blank list to later store data about each operation
operations = []

# Creating blank dataframes for later store return from pairs
first_traininig = int(periods.iloc[0, 3]) # removes first semester as it is formation period

Rpair = np.zeros((days-first_traininig, no_pairs))

Rp_ew_cc = pd.DataFrame(np.zeros((days-first_traininig, 2)),
                        columns=['Return', 'Semester'])

Rp_vw_fi = pd.DataFrame(np.zeros((days-first_traininig, 2)),
                        columns=['Return', 'Semester'])

ret_acum_df = pd.DataFrame(
    np.zeros((days-first_traininig, 4)), columns=['CC', 'FI', 'RMRF', 'SEMESTER'])

RmRf = np.zeros((days-first_traininig, 1))

risk_free = pd.DataFrame(np.zeros((days-first_traininig, 2)),
                        columns=['Return', 'Semester'])

In [6]:
risk_free

,Return,Semester
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
6168,0.0,0.0
6169,0.0,0.0
6170,0.0,0.0
6171,0.0,0.0


In [7]:
periods_with_open_pair = 0  # number of periods with pairs opened
periods_without_open_pair = 0  # number of periods without pairs opened
pairs_number = 0
pair_open = 0
days_open = np.zeros((no_pairs*10000, 1)) # measures number of days each pair open
no_pairs_opened = np.zeros((int(years*2-2), no_pairs)) # measures number of times pairs opened in each pair per 6 month period

counter = 0  # Keeps track of the days in the main loop

In [11]:
years = 2 # DOING THIS JUST TO LIMIT PERIODS

In [12]:
# ----------------------------------------------------
# Start of Main Loop - Creating Price Index
# ----------------------------------------------------
# Main part of the program starts here
# ----------------------------------------------------

big_loop = 0
i = 0

while big_loop < (years * 2 - 2): # years * 2 = no of semesters, -2 that are formational period
    twelve_months = int(periods.iloc[big_loop, 3])
    six_months = int(periods.iloc[big_loop + 2, 0])

    print(
        f"big_loop: {big_loop}, i: {i}, twelve_months: {twelve_months}, six_months: {six_months}")

    # ----------------------------------------------------
    # Create price index IPt by setting first Pt>0 to 1
    # ----------------------------------------------------

    # Preallocate a zeros matrix with the size of the Formation + Trading period
    # IPt = Indexed Price at time t
    IPt = np.zeros((int(twelve_months + six_months), num_assets))

    print("Generating Assets Price Index") 

    # below creates a table with prices indexed, start day 1 price 1, then day 2 price 1 * return
    for j in tqdm(range(0, num_assets)): # progress bar with number of assets
        m = 0
        for i2 in range(0, int(twelve_months+six_months)):  # same here
            if not math.isnan(Pt.iloc[i+i2, j]) and m == 0:
                IPt[i2, j] = 1
                m = 1
            elif not math.isnan(Pt.iloc[i+i2, j]) and m == 1:
                IPt[i2, j] = IPt[i2-1, j] * (1 + Rt.iloc[i+i2, j])

    pd.DataFrame(IPt).to_csv("IPt.csv", header=None, index=False, na_rep='NULL')

    
    listed1 = IPt[0, :] > 0  # Listed at the beginning (1xN vector of booleans)
    listed2 = IPt[int(twelve_months+six_months*(s1218 == 1))-1, :] > 0 # listed at the end: 12/18 months from now (1xN vector of booleans)
    listed = np.multiply(listed1, listed2)

    listed_num = np.sum(listed)
    listed_indexes = np.where(listed > 0)[0]
    listed_stocks = Pt.columns[listed_indexes]

    [D, ia, ib] = np.intersect1d(
        ticker2.iloc[:, big_loop], ticker2.iloc[:, big_loop+1], return_indices=True)

    ic = np.isin(D, ticker2.iloc[:, big_loop+2])

    Dic_unique_sorted, B_idx = np.unique(D[ic], return_index=True)

    listed_union = np.intersect1d(listed_stocks, Dic_unique_sorted)

    index_listed2 = [Pt.columns.get_loc(i) for i in listed_union if i in Pt]
    index_listed2.sort()

    no_listed2 = len(index_listed2)
    print(f"no_listed2: {no_listed2}")
    
    # ----------------------------------------------------
    # Add filters (if needed)
    # ----------------------------------------------------
    # e.g. remove if liquidity below value X, the second listed stock series etc.
    # ----------------------------------------------------
    # Desc stat of the price series
    # ----------------------------------------------------
    no_comp = np.transpose(sum(np.transpose(IPt > 0)))

    print(f'Period {big_loop}')

    # ----------------------------------------------------
    # Calc SSEs
    # ----------------------------------------------------

    sse = np.zeros((no_listed2, no_listed2))
    for j in tqdm(range(0, no_listed2 - 1)):
        for k in range(j+1, no_listed2):
            sse[j, k] = sum(np.power(IPt[0:int(
                twelve_months), index_listed2[j]]-IPt[0:int(twelve_months), index_listed2[k]], 2))

    print(f"SSE shape: {np.shape(sse)}")
    pd.DataFrame(sse).to_csv("SSE.csv", header=[
        str(i) for i in index_listed2], index=False)

    # ----------------------------------------------------
    # Find min SSEs
    # ----------------------------------------------------

    max_SSE = np.nanmax(sse) + 1
    min_SSE = np.zeros((no_pairs, 1))
    pairs = []
    min_SSE_ro = np.zeros((1, no_pairs))
    min_SSE_co = np.zeros((1, no_pairs))

    for ii in range(0, no_pairs):
        t_SSE = max_SSE
        for k in range(0, no_listed2-1):
            for l in range(k+1, no_listed2):
                if sse[k, l] > 0 and sse[k, l] < t_SSE:
                    t_SSE = sse[k, l]  # new minimum found

        if t_SSE == max_SSE:
            print("Error")

        ro, co = np.where(sse == t_SSE)
        ro = ro[0]
        co = co[0]
        min_SSE[ii, 0] = sse[ro, co]
        pairs.append({"s1_col": int(ro), "s2_col": int(co),
                     "s1_ticker": Pt.columns[index_listed2[ro]], "s2_ticker": Pt.columns[index_listed2[co]]})
        sse[ro, co] = max_SSE  # prevent re-selection

    
    # ----------------------------------------------------
    # Calculate returns during the 6 month period
    # ----------------------------------------------------


    ################# Find here where to calculate H and then use this to open or close in the following
    
    count_temp = counter

    print(f"counter value: {counter} | counter_temp = {count_temp}")

    print(f"Portfolio period from days {i+twelve_months} to {i+twelve_months+six_months-1}")
    
    for p in range(0, no_pairs): # inside each big loop period, following calculations will be performed for each pair
        first_col = index_listed2[pairs[p]['s1_col']] #about first asset in pair
        second_col = index_listed2[pairs[p]['s2_col']] #about second asset in pair
        counter = count_temp
        pairs_opened = 0
        new_pairs_opened = 0
        lag = 0
        can_trade = True
        last_operation = 0

        std_limit = np.std(IPt[0:twelve_months, first_col] - IPt[0:twelve_months, second_col])  # standard deviation

        print(f"Std limit: {std_limit}")
        spread = IPt[0:twelve_months, first_col] - IPt[0:twelve_months, second_col]

        # Fixed volatility estimated in the 12 months period. Doing the calculation one pair at a time
        # Presets all variables for each pair
        Rcum = np.zeros((twelve_months, 1))
        counter_ret = 1
        Rcum_ret = [1]

        wi = []

        for j in range(i+twelve_months, i+twelve_months+six_months-1):  # inside each period and regarding each pair, it will be calculated at each day:
            # Defining the period as from the first day of the twe_month to the last day of the twe_month

            ### Including Hurst calculation
            def hurst(price, min_lag=2, max_lag=100):
                lags =np.arange(min_lag, max_lag + 1)
                sigmas = [np.std(np.substract(price[tau:], price[:,-tau])) for tau in lags]
                H, c = np.polyfit(np.log10(lags), np.log10(sigmas), 1)
                return H, c, lags, sigmas
            
            if daylag == 0:  # w/o one day delay
                if not can_trade:
                    if (last_operation == 1 and (IPt[j-i, first_col]-IPt[j-i, second_col]) <= closing_threshold*std_limit) or (last_operation == -1 and (IPt[j-i, first_col]-IPt[j-i, second_col]) >= -closing_threshold*std_limit):
                        can_trade = True

                if pairs_opened == -1:  # pairs opened: long 1st, short 2nd stock
                    # print("Pair is long on 1st")
                    # If a sign to open has been given, then calcule the returns

                    Rpair[counter, p] = np.multiply(Rt.iloc[j, first_col], wi[0]) - np.multiply(Rt.iloc[j, second_col], wi[1]) # Rpair is the return of each pair.
                    Rcum[counter_ret, 0] = Rpair[counter, p]
                    counter_ret = counter_ret + 1
                    Rcum_ret = np.cumprod(1+Rcum)
                    lag = lag + 1  # used for paying tc

                    if wi_update == 1:  # The weight of each asset in the pair is updated.
                        wi[0] = wi[0]*(1+Rt.iloc[j, first_col])
                        wi[1] = wi[1]*(1+Rt.iloc[j, second_col])

                elif pairs_opened == 1:  # pairs opened: short 1st, long 2nd stock
                    Rpair[counter, p] = np.multiply(
                        -Rt.iloc[j, first_col], wi[0]) + np.multiply(Rt.iloc[j, second_col], wi[1])
                    Rcum[counter_ret, 0] = Rpair[counter, p]
                    counter_ret = counter_ret + 1
                    Rcum_ret = np.cumprod(1+Rcum)
                    lag = lag + 1

                    if wi_update == 1:
                        wi[0] = wi[0]*(1+Rt.iloc[j, first_col])
                        wi[1] = wi[1]*(1+Rt.iloc[j, second_col])
                        # print(f"Updated weights w1 {wi[0]} - w2 {wi[1]}")

                else:
                    Rpair[counter, p] = 0  # closed (this code not necessary)

                if ((pairs_opened == 1 and (IPt[j-i, first_col]-IPt[j-i, second_col]) <= closing_threshold*std_limit) or (counter_ret > duration_limit) or (Rcum_ret[-1] < Stop_loss) or (Rcum_ret[-1] >= Stop_gain)
                        or (pairs_opened == -1 and (IPt[j-i, first_col]-IPt[j-i, second_col]) >= -closing_threshold*std_limit)) and ((trade_req + (Vt.iloc[j, first_col] > 0) + (Vt.iloc[j, second_col] > 0)) > 1):
                    #print(f"Closing position {pairs_opened}. Price diff: {IPt[j-i, first_col]-IPt[j-i, second_col]}")
                    
                    converged = True

                    if Rcum_ret[-1] < Stop_loss:
                        Rcum_ret[-1] = Stop_loss
                        converged = False
                        can_trade = False
                        last_operation = pairs_opened

                    if counter_ret > duration_limit:
                        converged = False
                        can_trade = False
                        last_operation = pairs_opened

                    pairs_opened = 0  # close pairs: prices cross
                    # when pair is closed reset lag (it serves for paying tc)
                    lag = 0
                    # add a marker for closing used to calc length of the "open-period"
                    avg_price_dev[counter, no_pairs+p] = 1
                    # Includes trading cost in the last day of trading, due to closing position
                    Rpair[counter, p] = Rpair[counter, p] - percentage_costs

                    print(
                        f"Pair {pairs[p]['s1_ticker']}-{pairs[p]['s2_ticker']} closed. Days: {counter_ret} | Return: {Rcum_ret[-1]}")
                    operations.append({
                        "Semester": big_loop,
                        "Days": counter_ret,
                        "S1": pairs[p]['s1_ticker'],
                        "S2": pairs[p]['s2_ticker'],
                        "Pair": f"{pairs[p]['s1_ticker']}-{pairs[p]['s2_ticker']}",
                        "Return": Rcum_ret[-1],
                        "Converged": converged,
                        "Count day": counter
                    })

                    counter_ret = 1

                elif can_trade and (pairs_opened == 0) and (+IPt[j-i, first_col]-IPt[j-i, second_col] > opening_threshold*std_limit) and ((trade_req + (Vt.iloc[j, first_col] > 0) + (Vt.iloc[j, second_col] > 0)) > 1):
                    #print(f"Opening short position. Price diff: {IPt[j-i, first_col]-IPt[j-i, second_col]} | Threshold: {1.5*std_limit}")
                    if pairs_opened == 0:  # record dev (and time) at open
                        Rcum = np.zeros((six_months, 1))
                        counter_ret = 1
                        avg_price_dev[counter, p] = 2*(+IPt[j-i, first_col] - IPt[j-i, second_col])/(
                            IPt[j-i, first_col] + IPt[j-i, second_col])

                    # print(
                    #    f"Trading short pair {pairs[p]['s1_ticker']}-{pairs[p]['s2_ticker']} on day {j}")
                    pairs_opened = 1  # open pairs
                    lag = lag + 1  # - Lag was 0. On the next loop C will be paid
                    wi = [1, 1]

                elif can_trade and (pairs_opened == 0) and (IPt[j-i, first_col]-IPt[j-i, second_col] < -opening_threshold*std_limit) and ((trade_req + (Vt.iloc[j, first_col] > 0) + (Vt.iloc[j, second_col] > 0)) > 1):
                    #print(f"Opening long position. Price diff: {IPt[j-i, first_col]-IPt[j-i, second_col]} | Threshold: {-1.5*std_limit}")
                    if pairs_opened == 0:  # record dev (and time) at open
                        Rcum = np.zeros((six_months, 1))
                        counter_ret = 1
                        avg_price_dev[counter, p] = 2*(-IPt[j-i, first_col] + IPt[j-i, second_col])/(
                            IPt[j-i, first_col] + IPt[j-i, second_col])
                    # print(
                    #    f"Trading long pair {pairs[p]['s1_ticker']}-{pairs[p]['s2_ticker']} on day {j}")
                    pairs_opened = -1  # open pairs
                    lag = lag + 1
                    wi = [1, 1]

                counter += 1

            elif daylag == 1:
                if pairs_opened == -1:  # pairs opened: long 1st, short 2nd stock
                    Rpair[counter, p] = (+Rt.iloc[j, first_col] * wi[0] -
                                         Rt.iloc[j, second_col] * wi[1]) - (lag == 2)*trading_costs
                    Rcum[counter_ret, 0] = Rpair[counter, p]
                    counter_ret = counter_ret + 1
                    Rcum_ret = np.cumprod(1+Rcum)
                    if wi_update == 1:
                        wi[0] = wi[0]*(1+Rt.iloc[j, first_col])
                        wi[1] = wi[1]*(1+Rt.iloc[j, second_col])

                elif pairs_opened == 1:  # pairs opened: short 1st, long 2nd stock
                    Rpair[counter, p] = (-Rt.iloc[j, first_col] * wi[0] +
                                         Rt.iloc[j, second_col] * wi[1]) - (lag == 2)*trading_costs
                    Rcum[counter_ret, 0] = Rpair[counter, p]
                    counter_ret = counter_ret + 1
                    Rcum_ret = np.cumprod(1+Rcum)
                    if wi_update == 1:
                        wi[0] = wi[0]*(1+Rt.iloc[j, first_col])
                        wi[1] = wi[1]*(1+Rt.iloc[j, second_col])

                else:
                    Rpair[counter, p] = 0  # closed (this code not necessary)

                pairs_opened = new_pairs_opened

                if (pairs_opened == +1 and (IPt[j-i, first_col]-IPt[j-i, second_col]) <= 0
                    or Rcum_ret[-1] <= Stop_loss
                    or (Rcum_ret[-1] >= Stop_gain)
                        or pairs_opened == -1 and (IPt[j-i, first_col]-IPt[j-i, second_col]) >= 0) and ((trade_req + (Vt.iloc[j, first_col] > 0) + (Vt.iloc[j, second_col] > 0)) > 1):

                    new_pairs_opened = 0  # close prices: prices cross
                    # If the pairs are open and the spread is smaller than the
                    # threshold, close the position
                    lag = 0
                    # see above, marker
                    avg_price_dev[counter+1, no_pairs+p] = 1

                    if wi_update == 1:
                        Rpair[counter, p] = Rpair[counter, p] - \
                            trading_costs - percentage_costs

                elif (+IPt[j-i, first_col]-IPt[j-i, second_col] > 2.0*std_limit) and ((trade_req + (Vt.iloc[j, first_col] > 0) + (Vt.iloc[j, second_col] > 0)) > 1):
                    new_pairs_opened = 1  # open pairs
                    # If the difference between the prices are larger than
                    # the limit, and there is volume, open the position (short 1st, long 2nd)
                    lag = lag + 1
                    if pairs_opened == 0:
                        Rcum = np.zeros((six_months, 1))
                        counter_ret = 1

                elif (-IPt[j-i, first_col]+IPt[j-i, second_col] > 2.0*std_limit) and ((trade_req + (Vt.iloc[j, first_col] > 0) + (Vt.iloc[j, second_col] > 0)) > 1):
                    new_pairs_opened = -1  # open pairs
                    # If the difference between the prices are larger than
                    # the limit, and there is volume, open the position (short 2nd, long 1st)
                    lag = lag + 1
                    if pairs_opened == 0:  # - If the pair was closed, reset accumulated return matrix and counter
                        Rcum = np.zeros((six_months, 1))
                        counter_ret = 1

                if new_pairs_opened == +1 and lag == 1:
                    avg_price_dev[counter, p] = 2*(+IPt[j-i, first_col] - IPt[j-i, second_col])/(
                        IPt[j-i, first_col] + IPt[j-i, second_col])
                    lag = lag + 1
                    wi = [1, 1]

                elif new_pairs_opened == -1 and lag == 1:

                    avg_price_dev[counter, p] = 2*(-IPt(j-i, first_col) + IPt(
                        j-i, second_col))/(IPt(j-i, first_col) + IPt(j-i, second_col))
                    lag = lag + 1
                    wi = [1, 1]

                counter += 1

        if pairs_opened != 0:
            print(
                f"Pair {pairs[p]['s1_ticker']}-{pairs[p]['s2_ticker']} did not converged, Days: {counter_ret} | Return: {Rcum_ret[-1]}")
            # Includes trading cost in the last day of trading, due to closing position
            Rpair[counter-1, p] = Rpair[counter-1, p] - \
                trading_costs - percentage_costs
            avg_price_dev[counter-1, no_pairs+p] = 1
            operations.append(
                {
                    "Semester": big_loop,
                    "Days": counter_ret,
                    "S1": pairs[p]['s1_ticker'],
                    "S2": pairs[p]['s2_ticker'],
                    "Pair": f"{pairs[p]['s1_ticker']}-{pairs[p]['s2_ticker']}",
                    "Return": Rcum_ret[-1],
                    "Converged": False,
                    "Count day": counter
                }
            )

    # ------------------------------------------------
    # Finishing return and calculations
    # ------------------------------------------------
    
    pd.DataFrame(Rpair).to_csv("Rpair.csv", header=None, index=False)

            
    
    # ------------------------------------------------
    # Finishing big loop
    # ------------------------------------------------
    i = i + periods.iloc[big_loop, 0]
    big_loop = big_loop + 1

big_loop: 0, i: 0, twelve_months: 252, six_months: 128
Generating Assets Price Index


100%|███████████████████████████████████████████████████████████████████████████████| 1100/1100 [00:26<00:00, 41.40it/s]


no_listed2: 482
Period 0


100%|████████████████████████████████████████████████████████████████████████████████| 481/481 [00:04<00:00, 112.96it/s]


SSE shape: (482, 482)
counter value: 379 | counter_temp = 379
Portfolio period from days 252 to 379
Std limit: 3.0474365953173455e-05
Pair JCI-1436513D closed. Days: 33 | Return: 1.0000339721296883
Pair JCI-1436513D closed. Days: 13 | Return: 1.000067053695677
Pair JCI-1436513D closed. Days: 28 | Return: 1.0000880773032392
Pair JCI-1436513D closed. Days: 9 | Return: 1.0000487634419453
Pair JCI-1436513D closed. Days: 17 | Return: 1.0000517140747271
Pair JCI-1436513D closed. Days: 2 | Return: 1.0000496473936427
Pair JCI-1436513D closed. Days: 4 | Return: 1.0000530726014745
Pair JCI-1436513D closed. Days: 2 | Return: 1.0000530720456642
Pair JCI-1436513D closed. Days: 8 | Return: 1.0000303561700472
Std limit: 0.01706625926722224
Pair PEG-0961514D closed. Days: 10 | Return: 1.0277667832540782
Pair PEG-0961514D closed. Days: 15 | Return: 1.0124723024982716
Pair PEG-0961514D closed. Days: 6 | Return: 1.0190905987796226
Pair PEG-0961514D did not converged, Days: 34 | Return: 0.9900946246279267

/tmp/ipykernel_2074/1820846824.py:35: RuntimeWarning: invalid value encountered in scalar multiply0:19<00:04, 89.46it/s]
  IPt[i2, j] = IPt[i2-1, j] * (1 + Rt.iloc[i+i2, j])
100%|███████████████████████████████████████████████████████████████████████████████| 1100/1100 [00:26<00:00, 40.97it/s]


no_listed2: 488
Period 1


100%|████████████████████████████████████████████████████████████████████████████████| 487/487 [00:04<00:00, 119.12it/s]


SSE shape: (488, 488)
counter value: 506 | counter_temp = 506
Portfolio period from days 380 to 505
Std limit: 3.3031880565138366e-05
Pair JCI-1436513D closed. Days: 6 | Return: 1.0000786378508644
Pair JCI-1436513D closed. Days: 32 | Return: 1.0000616724447267
Pair JCI-1436513D closed. Days: 15 | Return: 1.000032219174004
Pair JCI-1436513D closed. Days: 63 | Return: 1.000035465152786
Pair JCI-1436513D did not converged, Days: 7 | Return: 0.9999536509536028
Std limit: 0.020027840960041322
Pair EIX-NEE closed. Days: 14 | Return: 0.95
Pair EIX-NEE did not converged, Days: 1 | Return: 0.95
Std limit: 0.020155035115615737
Pair CEG-NEE closed. Days: 5 | Return: 1.0207794972501811
Pair CEG-NEE closed. Days: 2 | Return: 1.0185252305834318
Pair CEG-NEE closed. Days: 2 | Return: 1.0194295952967976
Pair CEG-NEE closed. Days: 17 | Return: 1.0266101966863068
Pair CEG-NEE closed. Days: 47 | Return: 1.0364685624452616
Pair CEG-NEE closed. Days: 2 | Return: 1.020366247497053
Std limit: 0.0206703510584